In [1]:
import numpy as np
from sklearn.datasets.samples_generator import make_blobs
from sklearn.metrics.pairwise import pairwise_distances
from matplotlib import pyplot as plt
import pandas as pd

from imblearn.datasets import make_imbalance

from lsh_functions import *

rng = np.random.RandomState(42)

# PLSH 2-layer small example

### Create a clustering instance - set of points with centers

In [2]:
n = 100000
k = 100
d = 50

In [3]:
# sigma = 1
# a = 0.5 # (also epsilon)
# c = np.power(np.power(a, 2) * np.power(np.log(n), 3), 0.5)
# t = int(np.power(np.log(n), 0.5))
# w = t * sigma * np.power(4*np.log(n), 0.5)
# n_hashes = int(2/a)

# print c
# print t 
# print w
# print n_hashes

In [4]:
sigma = 1
# a = 0.5 # (also epsilon)

t = int(np.ceil(np.log(n)/float(np.power(np.log(np.log(n)), 2))))
# w = 8 * sigma *np.power(np.log(n), 1.5)
w = 10
# c = 4 * w / float(np.power(t, 0.5))
c = 20
n_hashes = int(np.ceil(np.power(np.log(np.log(n)), 2)))

print 'c: ', c
print 't: ', t 
print 'w: ', w
print 'n_hashes: ', n_hashes

c:  20
t:  2
w:  10
n_hashes:  6


In [5]:
# # parameters changed
# sigma = 1
# a = 0.5 # (also epsilon)
# c = np.power(np.power(a, 2) * np.power(np.log(n), 3), 0.5)
# t = int(np.power(np.log(n), 0.5))
# w = sigma * np.power(np.log(n), 1.5)
# n_hashes = int(np.log(n))

# print c
# print t 
# print w
# print n_hashes

#### PLSH parameters

In [6]:
multiplier = 3
print 't: ', t
print 'multiplier: ', multiplier
print 'n_grids: ', int(np.power(2, multiplier*t*np.log(t)))

t:  2
multiplier:  3
n_grids:  17


In [7]:
def create_ratios(n, k):
    a = rng.rand(k)
    m = n/np.sum(a)
    ratios = a*m
    ratios = np.array(ratios, dtype=int)
    ratios[np.argmin(ratios)] += 1
    diff = n - np.sum(ratios)
    ratios[np.argmax(ratios)] += + diff
    return ratios

ratios = create_ratios(n, k)

centers = generate_centers(n_centers=k, dim=d, sep=c*sigma)

# set cluster std to for cluster radius = \sigma
cluster_std = sigma/(2*np.sqrt(d))

pts, clust_ids = make_blobs(ratios[0], d, centers=[centers[0]], cluster_std=cluster_std)

for i in range(1, k):
    pt, _ = make_blobs(ratios[i], d, centers=[centers[i]], cluster_std=cluster_std)
    pts = np.vstack((pts, pt))
    clust_ids = np.hstack((clust_ids, np.ones(ratios[i])*i))

rnd_order = rng.choice(np.arange(n), n, replace=False)
pts = pts[rnd_order]
clust_ids = np.array(clust_ids[rnd_order], dtype=int)

In [8]:
print 'diameter of centers: ', np.max(pairwise_distances(centers))

diameter of centers:  39.95690610766462


In [9]:
# # write data into files
# df_pts = pd.DataFrame(pts)
# df_clust_ids = pd.DataFrame(clust_ids)

# df_pts.to_csv('clustered_pts.txt', sep=' ', header=False, index=False)
# df_clust_ids.to_csv('cluster_ids.txt', header=False, index=False)

# df_one_pt = pd.DataFrame(np.array(df_pts.ix[3434]).reshape(1, 50))
# df_one_pt.to_csv('one_pt.txt', sep=' ', header=False, index=False)

### create grids, hash points and find plsh hashes

In [10]:
#create grids
shifts = generate_grids(t, multiplier=3)
print 'created grids'

# create hash matrices
hash_matrices = []
for i in range(n_hashes):
    hash_matrices += [get_hash_matrix(t, d)]
print 'created hash matrices'

# hash pts into t dimension
hashed_values = []
for i in range(n_hashes):
    hashed_values += [np.dot(hash_matrices[i], pts.transpose()).transpose()]
print 'hashed to t dimension'

print "finding plsh"

# find plsh of pts
full_hashes = []
for i in range(n):
    hashed_pts = []
    for j in range(n_hashes):
        hashed_pts += [hashed_values[j][i]]
    full_hashes += [multi_hash(hashed_pts, shifts, w)]
    if i%10000 == 0:
        print str(i), ' done'
        print full_hashes[i]

# first level hash - plsh
full_hashes = np.array(full_hashes)

created grids
created hash matrices
hashed to t dimension
finding plsh
0  done
3121-1-30-200000-1-11-11
10000  done
1-101-1-30-3-160-17-1-26-11
20000  done
3020-1-31-3-201-11-1-3303
30000  done
0-111-1-31-3-160-1500202
40000  done
7-102-2-21-3-10001-1-2303
50000  done
0-113-1-21-4-231010-32-13
60000  done
1-101-1-30-3-160-17-1-26-11
70000  done
3121-1-32-200000-1-11-11
80000  done
0015-1-30-3-131030-11-10
90000  done
15-120-1-31-3-201-11-1-3303


### Evals

In [11]:
def n_hashes_per_cluster(clust_ids, full_hashes):
    clusters = np.unique(clust_ids)
    n_hashes_counts = []
    for i in range(clusters.shape[0]):
        n_hashes_counts += [np.unique(full_hashes[np.where(clust_ids==clusters[i])[0]]).shape[0]]
    return clusters, n_hashes_counts

In [12]:
def clusters_in_buckets(clust_ids, full_hashes):
    buckets = np.unique(full_hashes)
    clusters = []
    for i in range(buckets.shape[0]):
        clusters += [clust_ids[np.where(full_hashes==buckets[i])[0]]]
    return buckets, clusters

In [13]:
clusters, hash_counts = n_hashes_per_cluster(clust_ids, full_hashes)
print 'avg # of buckets per cluster: ', np.mean(hash_counts)

avg # of buckets per cluster:  7.97


In [14]:
buckets, clusters = clusters_in_buckets(clust_ids, full_hashes)
print 'total # of buckets: ', buckets.shape[0]

bucket_stats = []
for i in range(len(clusters)):
    a, b = np.unique(clusters[i], return_counts=True)    
    max_c_id = np.argmax(b)
    max_c = a[max_c_id]
    n_clusters = a.shape[0]
    bucket_stats += [[n_clusters, max_c]]
bucket_stats = np.array(bucket_stats)

#print bucket_stats
print 'avg # of clusters in a bucket: ', np.mean(bucket_stats[:, 0])

total # of buckets:  797
avg # of clusters in a bucket:  1.0


In [15]:
# avg # of pts in a bucket
distinct_buckets = np.unique(full_hashes)
bucket_counts = []
for val in distinct_buckets:
    bucket_counts += [np.where(full_hashes==val)[0].shape[0]]
print 'avg pts in a bucket: ', np.mean(bucket_counts)
print 'max pts in a bucket: ', np.max(bucket_counts)
print 'min pts in a bucket: ', np.min(bucket_counts)

avg pts in a bucket:  125.47051442910916
max pts in a bucket:  1898
min pts in a bucket:  1


### second hash

In [16]:
L = 5
n_groups = L*k

In [17]:
second_hashes = []
for i in range(len(full_hashes)):
    second_hashes += [second_hash(full_hashes[i], n_groups)]

In [18]:
distinct_groups = np.unique(second_hashes)
group_counts = []
for val in distinct_groups:
    group_counts += [np.where(second_hashes==val)[0].shape[0]]
print 'avg pts in a group: ', np.mean(group_counts)
print 'max pts in a group: ', np.max(group_counts)
print 'min pts in a group: ', np.min(group_counts)

avg pts in a group:  245.09803921568627
max pts in a group:  1898
min pts in a group:  1


## Algorithm